Import Libraries....

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Load Data.....

In [3]:
df = pd.read_csv('/content/ncr_ride_bookings.csv')
print(df.head())

         Date      Time    Booking ID   Booking Status   Customer ID  \
0  2024-03-23  12:29:38  "CNR5884300"  No Driver Found  "CID1982111"   
1  2024-11-29  18:01:39  "CNR1326809"       Incomplete  "CID4604802"   
2  2024-08-23  08:56:10  "CNR8494506"        Completed  "CID9202816"   
3  2024-10-21  17:17:25  "CNR8906825"        Completed  "CID2610914"   
4  2024-09-16  22:08:00  "CNR1950162"        Completed  "CID9933542"   

    Vehicle Type      Pickup Location      Drop Location  Avg VTAT  Avg CTAT  \
0          eBike          Palam Vihar            Jhilmil       NaN       NaN   
1       Go Sedan        Shastri Nagar  Gurgaon Sector 56       4.9      14.0   
2           Auto              Khandsa      Malviya Nagar      13.4      25.8   
3  Premier Sedan  Central Secretariat           Inderlok      13.1      28.5   
4           Bike     Ghitorni Village        Khan Market       5.3      19.6   

   ...  Reason for cancelling by Customer Cancelled Rides by Driver  \
0  ...         

In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 21 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Date                               150000 non-null  object 
 1   Time                               150000 non-null  object 
 2   Booking ID                         150000 non-null  object 
 3   Booking Status                     150000 non-null  object 
 4   Customer ID                        150000 non-null  object 
 5   Vehicle Type                       150000 non-null  object 
 6   Pickup Location                    150000 non-null  object 
 7   Drop Location                      150000 non-null  object 
 8   Avg VTAT                           139500 non-null  float64
 9   Avg CTAT                           102000 non-null  float64
 10  Cancelled Rides by Customer        10500 non-null   float64
 11  Reason for cancelling by Customer  1050

Parse Date and Time columns

In [6]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# keep time column as it is, but clean formatting if nedded
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce')
display(df[['Date', 'Time']].head())

,Date,Time
0,2024-03-23,1900-01-01 12:29:38
1,2024-11-29,1900-01-01 18:01:39
2,2024-08-23,1900-01-01 08:56:10
3,2024-10-21,1900-01-01 17:17:25
4,2024-09-16,1900-01-01 22:08:00


Remove quotes from IDs

In [9]:
df['Booking ID'] = df['Booking ID'].str.replace('"', '')
df['Customer ID'] = df['Customer ID'].str.replace('"', '')
display(df[['Booking ID', 'Customer ID']].head())

,Booking ID,Customer ID
0,CNR5884300,CID1982111
1,CNR1326809,CID4604802
2,CNR8494506,CID9202816
3,CNR8906825,CID2610914
4,CNR1950162,CID9933542


filling missing numeric values(using median)

In [14]:
numeric_cols = ['Avg VTAT', 'Avg CTAT', 'Booking Value', 'Ride Distance', 'Driver Ratings', 'Customer Rating']
for col in numeric_cols:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median())
display(df[numeric_cols].head())

,Avg VTAT,Avg CTAT,Booking Value,Ride Distance,Driver Ratings,Customer Rating
0,8.3,28.8,414.0,23.72,4.3,4.5
1,4.9,14.0,237.0,5.73,4.3,4.5
2,13.4,25.8,627.0,13.58,4.9,4.9
3,13.1,28.5,416.0,34.02,4.6,5.0
4,5.3,19.6,737.0,48.21,4.1,4.3


Fill missing categorical values

In [20]:
df['Payment Method'] = df['Payment Method'].fillna('Unknown')
df['Booking Status'] = df['Booking Status'].fillna('Unknown')

categorical_cancel_cols = ['Reason for cancelling by Customer', 'Driver Cancellation Reason', 'Incomplete Rides Reason']
for col in categorical_cancel_cols:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown')
        display(df[col].value_counts())

,count
Reason for cancelling by Customer,
Unknown,139500
Wrong Address,2362
Change of plans,2353
Driver is not moving towards pickup location,2335
Driver asked to cancel,2295
AC is not working,1155


,count
Driver Cancellation Reason,
Unknown,123000
Customer related issue,6837
The customer was coughing/sick,6751
Personal & Car related issues,6726
More than permitted people in there,6686


,count
Incomplete Rides Reason,
Unknown,141000
Customer Demand,3040
Vehicle Breakdown,3012
Other Issue,2948


In [21]:
numeric_cancel_cols = ['Cancelled Rides by Customer', 'Cancelled Rides by Driver', 'Incomplete Rides']
for col in numeric_cancel_cols:
    if col in df.columns:
        df[col] = df[col].fillna(0)
        display(df[col].value_counts())

,count
Cancelled Rides by Customer,
0.0,139500
1.0,10500


,count
Cancelled Rides by Driver,
0.0,123000
1.0,27000


,count
Incomplete Rides,
0.0,141000
1.0,9000


Drop rows where critical fields are missing

In [41]:
df = df.dropna(subset=['Date', 'Booking ID'])

Remove Duplicate Rows

In [42]:
print("Number of rows before removing duplicates:", len(df))
df.drop_duplicates(inplace=True)
print("Number of rows after removing duplicates:", len(df))

Number of rows before removing duplicates: 57132
Number of rows after removing duplicates: 57132


Add Derived Columns for Analysis

In [30]:
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Day of Week'] = df['Date'].dt.day_name()
df['Hour'] = df['Time'].dt.hour       #Extract hour for time-based analysis

Outlier Detection using IQR for key numeric fields

In [43]:
outlier_cols = ['Booking Value', 'Ride Distance', 'Driver Ratings', 'Customer Rating']
print("Number of rows before removing outliers:", len(df))
for col in outlier_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
print("Number of rows after removing outliers:", len(df))
display(df.head())

Number of rows before removing outliers: 57132
Number of rows after removing outliers: 48000


,Date,Time,Booking ID,Booking Status,Customer ID,Vehicle Type,Pickup Location,Drop Location,Avg VTAT,Avg CTAT,...,Incomplete Rides Reason,Booking Value,Ride Distance,Driver Ratings,Customer Rating,Payment Method,Month,Day,Day of Week,Hour
0,2024-03-23,1900-01-01 12:29:38,CNR5884300,No Driver Found,CID1982111,eBike,Palam Vihar,Jhilmil,8.3,28.8,...,Unknown,414.0,23.72,4.3,4.5,Unknown,3,23,Saturday,12
2,2024-09-14,1900-01-01 12:49:09,CNR4510807,No Driver Found,CID7873618,Go Sedan,Noida Sector 62,Noida Sector 18,8.3,28.8,...,Unknown,414.0,23.72,4.3,4.5,Unknown,9,14,Saturday,12
4,2024-09-18,1900-01-01 08:09:38,CNR9551927,No Driver Found,CID7568143,Auto,Vidhan Sabha,AIIMS,8.3,28.8,...,Unknown,414.0,23.72,4.3,4.5,Unknown,9,18,Wednesday,8
5,2024-06-25,1900-01-01 22:44:15,CNR4386945,Cancelled by Driver,CID5543520,eBike,Patel Chowk,Kherki Daula Toll,4.6,28.8,...,Unknown,414.0,23.72,4.3,4.5,Unknown,6,25,Tuesday,22
6,2024-12-15,1900-01-01 15:08:25,CNR6739317,Cancelled by Driver,CID8682675,Go Sedan,Vinobapuri,GTB Nagar,6.0,28.8,...,Unknown,414.0,23.72,4.3,4.5,Unknown,12,15,Sunday,15


Save cleaned Data

In [45]:
df.to_csv('/content/ncr_ride_bookings_final_cleaned.csv', index=False)
print("Final Cleaned Dataset Saved.")

Final Cleaned Dataset Saved.


EDA